In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import (
    CSVLogger,
    ModelCheckpoint,
    TensorBoard,
)
from keras.datasets import cifar10
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.optimizers.legacy import SGD

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

CIFAR-10 stands for Canadian Institute For Advanced Research 10, which is a dataset of 50,000 32x32 color images in 10 classes, with 6,000 images per class. The dataset is commonly used for image classification tasks in machine learning and computer vision.


The 10 classes in the CIFAR-10 dataset are:

1. airplane
2. automobile
3. bird
4. cat
5. deer
6. dog
7. frog
8. horse
9. ship
10. truck


In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

In [ ]:
def plot_sample(index):
    plt.figure(figsize=(15, 2))
    plt.imshow(X_train[index])
    plt.title("Label: {}".format(class_names[y_train[index][0]]))

In [ ]:
plot_sample(101)

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
ann = Sequential(
    layers=[
        Flatten(input_shape=(32, 32, 3), name="input_layer"),
        Dense(1600, activation="sigmoid", name="hidden_layer_1"),
        Dense(800, activation="sigmoid", name="hidden_layer_2"),
        Dense(400, activation="sigmoid", name="hidden_layer_3"),
        Dense(200, activation="sigmoid", name="hidden_layer_4"),
        Dense(100, activation="sigmoid", name="hidden_layer_5"),
        Dense(10, activation="softmax", name="output_layer"),
    ],
    name="ann",
)

In [ ]:
ann.layers

In [ ]:
ann.summary()

In [ ]:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
epochs = 10
ann.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
ann_checkpoint_callback = ModelCheckpoint(
    filepath="../Models/cifar10_ann", monitor="val_accuracy", save_best_only=True
)
ann_tensorboard_callback = TensorBoard(
    log_dir="../Logs/cifar10_ann_logs", histogram_freq=1, write_images=True
)
ann_csvlogger_callback = CSVLogger("../Logs/cifar10_ann_logs.csv")

ann_history = ann.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=epochs,
    validation_split=0.1,
    # callbacks=[
    #     ann_checkpoint_callback,
    #     ann_tensorboard_callback,
    #     ann_csvlogger_callback,
    # ],
)

In [ ]:
ann_train_loss = ann_history.history["loss"]
ann_train_acc = ann_history.history["accuracy"]
ann_val_acc = ann_history.history["val_accuracy"]
ann_val_loss = ann_history.history["val_loss"]

print("Training loss:", np.mean(ann_train_loss))
print("Validation loss:", np.mean(ann_val_loss))
print(
    "%f Training accuracy with a standard deviation of %f"
    % (np.mean(ann_train_acc), np.std(ann_train_acc))
)
print(
    "%f Validation accuracy with a standard deviation of %f"
    % (np.mean(ann_val_acc), np.std(ann_val_acc))
)

In [ ]:
epochs_range = range(epochs)

# Plot the training and validation accuracy
plt.figure(figsize=(12, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, ann_train_acc, label="Training Accuracy")
plt.plot(epochs_range, ann_val_acc, label="Validation Accuracy")
plt.legend(loc="upper left")
plt.title("Training and Validation Accuracy")

# Plot the training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, ann_train_loss, label="Training Loss")
plt.plot(epochs_range, ann_val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")

plt.savefig("../Images/cifar10_ann_acc_loss.png", bbox_inches="tight")

In [ ]:
ann_test_loss, ann_test_accuracy = ann.evaluate(X_test, y_test)
print("Test loss:", ann_test_loss)
print("Test accuracy:", ann_test_accuracy)

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
cnn = Sequential(
    layers=[
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            input_shape=(32, 32, 3),
            name="conv_layer_1",
        ),
        Conv2D(
            filters=32,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            name="conv_layer_2",
        ),
        MaxPooling2D((2, 2), name="pooling_layer_1"),
        Dropout(0.2, name="dropout_layer_1"),
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            name="conv_layer_3",
        ),
        Conv2D(
            filters=64,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            name="conv_layer_4",
        ),
        MaxPooling2D((2, 2), name="pooling_layer_2"),
        Dropout(0.2, name="dropout_layer_2"),
        Conv2D(
            filters=128,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            name="conv_layer_5",
        ),
        Conv2D(
            filters=128,
            kernel_size=(3, 3),
            activation="relu",
            padding="same",
            kernel_initializer="he_uniform",
            name="conv_layer_6",
        ),
        MaxPooling2D((2, 2), name="pooling_layer_3"),
        Dropout(0.2, name="dropout_layer_3"),
        Flatten(name="flatten_layer"),
        Dense(128, activation="relu", name="dense_layer_1"),
        Dropout(0.2, name="dropout_layer_4"),
        Dense(10, activation="softmax", name="final_layer"),
    ],
    name="cnn",
)

In [ ]:
cnn.layers

In [ ]:
cnn.summary()

In [ ]:
optimizer = SGD(lr=0.001, momentum=0.9)

In [ ]:
epochs = 10

In [ ]:
cnn.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
cnn_checkpoint_callback = ModelCheckpoint(
    filepath="../Models/cifar10_cnn",
    monitor="val_accuracy",
    save_best_only=True,
)
cnn_tensorboard_callback = TensorBoard(
    log_dir="../Logs/cifar10_cnn_logs", histogram_freq=1, write_images=True
)
cnn_csvlogger_callback = CSVLogger("../Logs/cifar10_cnn_logs.csv")
cnn_history = cnn.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=epochs,
    validation_split=0.1,
    # callbacks=[cnn_checkpoint_callback, cnn_tensorboard_callback],
)

In [ ]:
cnn_train_acc = cnn_history.history["accuracy"]
cnn_val_acc = cnn_history.history["val_accuracy"]
cnn_train_loss = cnn_history.history["loss"]
cnn_val_loss = cnn_history.history["val_loss"]


print("Training loss:", np.mean(cnn_train_loss))
print("Validation loss:", np.mean(cnn_val_loss))
print(
    "%f Training accuracy with a standard deviation of %f"
    % (np.mean(cnn_train_acc), np.std(cnn_train_acc))
)
print(
    "%f Validation accuracy with a standard deviation of %f"
    % (np.mean(cnn_val_acc), np.std(cnn_val_acc))
)

In [ ]:
epochs_range = range(epochs)

# Plot the training and validation accuracy
plt.figure(figsize=(12, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, cnn_train_acc, label="Training Accuracy")
plt.plot(epochs_range, cnn_val_acc, label="Validation Accuracy")
plt.legend(loc="upper left")
plt.title("Training and Validation Accuracy")

# Plot the training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, cnn_train_loss, label="Training Loss")
plt.plot(epochs_range, cnn_val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training and Validation Loss")

plt.savefig("../Images/cifar10_cnn_acc_loss.png", bbox_inches="tight")

In [ ]:
cnn_test_loss, cnn_test_accuracy = cnn.evaluate(X_test, y_test)
print("Test loss:", cnn_test_loss)
print("Test accuracy:", cnn_test_accuracy)

In [ ]:
y_pred = cnn.predict(X_test)